# Machine Learning Engineering
# Project - stage II
**Authors:**
- Marcin Grabysz
- Aleksandra Jamróz

**Task**

Our song database is quite rich - they are described by many interesting parameters. Why nobody tagged if they are in major or minor scale so far? We have to change it!

In [1]:
import pandas as pd
import numpy as np

import xgboost as xgb

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, recall_score
from sklearn.preprocessing import StandardScaler

# Basic model

In our previous analysis we tried simple XGBoost approach. It will serve as our basic model.

In [2]:
# Data loading
# exactly the same as in stage I

df = pd.read_json("IUM23L_Zad_08_03_v2/tracks.jsonl", lines=True)
df.drop_duplicates(subset=("name", "id_artist"), inplace=True)
df = df.loc[df['mode'].notnull()]
df = df.drop(columns=["id", "name", "popularity", "explicit", "duration_ms", "id_artist", "release_date"])

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25688 entries, 0 to 25928
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      25688 non-null  float64
 1   energy            25688 non-null  float64
 2   key               25688 non-null  int64  
 3   mode              25688 non-null  float64
 4   loudness          25688 non-null  float64
 5   speechiness       25688 non-null  float64
 6   acousticness      25688 non-null  float64
 7   instrumentalness  25688 non-null  float64
 8   liveness          25688 non-null  float64
 9   valence           25688 non-null  float64
 10  tempo             25688 non-null  float64
 11  time_signature    25688 non-null  int64  
dtypes: float64(10), int64(2)
memory usage: 2.5 MB


In [3]:
import joblib
import sys

sys.modules['sklearn.externals.joblib'] = joblib

In [4]:
from sklearn.externals.joblib import dump, load

def split(df, file=None):
    # dividing into X and y 

    X_final = df.drop(columns=["mode"])
    y_final = df["mode"]

    # scaling the data and train test split

    scaler = StandardScaler()
    scaler.fit(X_final)
    if file:
        dump(scaler, file, compress=True)

    X_scaled = scaler.transform(X_final)
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_final, test_size=0.2, random_state=32)
    return X_train, X_test, y_train, y_test, X_scaled, y_final

In [5]:
def get_scores(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    acc = accuracy_score(y_test, pred)
    recall = recall_score(y_test, pred)
    print("Accuracy: ", round(acc*100, 4), "%")
    print("Recall: ", round(recall*100, 4), "%")

In [6]:
def get_cross_val_scores(model, X, y, folds=3):
    print("Cross validation scores: ", cross_val_score(model, X, y, cv=folds))

In [7]:
X_train, X_test, y_train, y_test, X_final, y_final = split(df, 'basic.bin')

In [8]:
xgb_tree = xgb.XGBClassifier()
# saving basic model:
xgb_tree.fit(X_train, y_train)
xgb_tree.save_model('basic_model.json')
get_scores(xgb_tree, X_train, y_train, X_test, y_test)
get_cross_val_scores(xgb.XGBClassifier(), X_final, y_final)

Accuracy:  69.3266 %
Recall:  87.7375 %
Cross validation scores:  [0.67569777 0.67896765 0.68698902]


In [9]:
# feature importances:

sorted(
    list(zip(df.columns, xgb_tree.feature_importances_)),
    key=lambda t: t[1],
    reverse=True
)

[('key', 0.28115368),
 ('speechiness', 0.08026361),
 ('acousticness', 0.07867877),
 ('loudness', 0.07853775),
 ('danceability', 0.07111322),
 ('liveness', 0.07092597),
 ('mode', 0.07055699),
 ('energy', 0.07052805),
 ('valence', 0.07020085),
 ('instrumentalness', 0.06541826),
 ('tempo', 0.06262291)]

Main column having impact on our prediction is key. It is not a surprise for us, as we conducted research in the first stage of the project and expected exactly this. Rest of the columns have similar impact on the model, starting with speechiness and ending with tempo. 

## Basic model - results

Data preprocession for basic model was not advanced. Steps which were taken:
- deletion of columns which we think are useless
- data scaling.

Nothing more was done. More steps will be introduced in further modelling.

Highest score so far is **69,33% accuracy**.

# Advanced model

We will try more developed approaches.
Plans:
- using logistic regressor instead of xgboost
- changing class balance (making in more 50/50 between categories)
- choosing appropriate columns
- modyfing column structure if needed
- hyperparameter tuning.

Following cells are just some of our approaches. Including all of the attempts would take too much space and affect the clarity of the report.


## Logistic regression

In [10]:
l_regresson = LogisticRegression()
get_scores(l_regresson, X_train, y_train, X_test, y_test)

Accuracy:  68.0226 %
Recall:  99.4531 %


In [11]:
l_regresson = LogisticRegression(C = 0.5)
get_scores(l_regresson, X_train, y_train, X_test, y_test)

Accuracy:  68.042 %
Recall:  99.4819 %


In [12]:
weights = {
    0: 0,
    1: 1.3
}

l_regresson = LogisticRegression(C = 0.5, class_weight=weights)
get_scores(l_regresson, X_train, y_train, X_test, y_test)

Accuracy:  67.6139 %
Recall:  100.0 %


After several experiments with following parameters:
- penalty
- tol
- C
- solver
- warm_start
- class_weights

only changing C value had positive impact of the score. Even though, the accuracy was lower than xgboost, so we resigned from further develpoment of logistic regressor.

## Balancing classes

66% of the dataset are major tracks, while the rest (34%) are minor tracks. Those classes are unbalanced and it shows in basic approach. As we checked in previous stage (copied from the analyses):

*Test set*

*Major tracks in test set:  3396.0 -> Part of dataset:  66.1 %*

*Minor tracks in test set:  1742.0 -> Part of dataset:  33.9 %*

*Prediction*

*Major tracks in prediction:  4203 -> Part of dataset:  81.8 %*

*Minor tracks in prediction:  935 -> Part of dataset:  18.2 %*

We can use this knowledge and try to improve the score. 

In [13]:
xgb_tree = xgb.XGBClassifier(scale_pos_weight = 1.01)
get_scores(xgb_tree, X_train, y_train, X_test, y_test)


Accuracy:  69.385 %
Recall:  88.0829 %


Adding *scale_pos_weight* parameter didn't improve the score much (0.05%). Let us try decreasing number of records of more classes in the dataset. 

In [14]:
df_0 = df[df['mode'] == 0]
df_1 = df[df['mode'] == 1]
diff = len(df_1.index) - len(df_0.index)
df_1 = df_1.sample(frac=1).iloc[diff:]
df_conc = pd.concat([df_1, df_0])
df_conc = df_conc.sample(frac=1)
df_conc.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 17210 entries, 19603 to 19693
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      17210 non-null  float64
 1   energy            17210 non-null  float64
 2   key               17210 non-null  int64  
 3   mode              17210 non-null  float64
 4   loudness          17210 non-null  float64
 5   speechiness       17210 non-null  float64
 6   acousticness      17210 non-null  float64
 7   instrumentalness  17210 non-null  float64
 8   liveness          17210 non-null  float64
 9   valence           17210 non-null  float64
 10  tempo             17210 non-null  float64
 11  time_signature    17210 non-null  int64  
dtypes: float64(10), int64(2)
memory usage: 1.7 MB


In [15]:
X_train, X_test, y_train, y_test, X_final, y_final = split(df_conc)

xgb_tree = xgb.XGBClassifier()
get_scores(xgb_tree, X_train, y_train, X_test, y_test)


Accuracy:  60.4881 %
Recall:  61.421 %


What you can see here is balancing the classes manually. In our case this move drastically decreased number of instances in the dataset. Client provided this dataset and no more rows are available, so we can't increase its size by loading more data. Cutting random rows to make number of instances of each class equal resulted with much smaller dataset, which was not enough for the model to learn.

## Changing 'key' column structure

Key column is now a numeric column. It shouldn't - key values are not linear and we can't sort them. First approach would be to create dummies out of this column.
 

In [16]:
df_dummies = pd.get_dummies(df, columns=['key'])
X_train, X_test, y_train, y_test, X_final, y_final = split(df_dummies)
xgb_tree = xgb.XGBClassifier()
get_scores(xgb_tree, X_train, y_train, X_test, y_test)


Accuracy:  69.0152 %
Recall:  87.3921 %


There are 12 different key values. It is not a small number, so our model doesn't cope with this type of data better than the standard one. We will try a different idea. We will change 'key' value to truly linear form. We will sort the keys by balance between minor and major tracks appearing in exact key. We hope that will affect our model to differentiate between keys containing more minor than major instances from those which contain a reversed balance.

In [17]:
letter_dict = {0: "C", 1: "C#", 2: "D", 3: "D#", 4: "E", 5: "F", 6: "F#", 7: "G", 8: "G#", 9: "A", 10: "A#", 11: "B"}
df["key"].replace(letter_dict, inplace=True)

def get_keys_mode_df():
    keys_dict = {}
    for index, row in df[["key", "mode"]].iterrows():
        if row["key"] not in keys_dict.keys():
            keys_dict[row["key"]] = {"major": 0, "minor": 0, "balance": 0}
        if row["mode"] == 0:
            keys_dict[row["key"]]["minor"] += 1
        if row["mode"] == 1:
            keys_dict[row["key"]]["major"] += 1
    for key in keys_dict:
        keys_dict[key]['balance'] = keys_dict[key]['major'] / keys_dict[key]['minor'] 
    return pd.DataFrame.from_dict(keys_dict), keys_dict

df_key, keys_dict = get_keys_mode_df()
df_key


,F#,C,D,C#,A#,G#,E,G,B,A,F,D#
major,616.000000,2630.000000,2252.000000,1182.00000,938.000000,1162.00000,1081.000000,2552.000000,681.000000,1918.000000,1418.000000,653.000000
minor,639.000000,691.000000,722.000000,539.00000,592.000000,287.00000,1135.000000,724.000000,1021.000000,1169.000000,852.000000,234.000000
balance,0.964006,3.806078,3.119114,2.19295,1.584459,4.04878,0.952423,3.524862,0.666993,1.640719,1.664319,2.790598


In [18]:

keys = []
balances = []
for key in keys_dict:
    keys.append(key)
    balances.append(keys_dict[key]['balance'])

sorted_keys = sorted(
    list(zip(keys, balances)),
    key=lambda t: t[1]
)

sorted_keys

[('B', 0.6669931439764937),
 ('E', 0.9524229074889868),
 ('F#', 0.9640062597809077),
 ('A#', 1.5844594594594594),
 ('A', 1.6407185628742516),
 ('F', 1.664319248826291),
 ('C#', 2.1929499072356213),
 ('D#', 2.7905982905982905),
 ('D', 3.119113573407202),
 ('G', 3.5248618784530388),
 ('C', 3.806078147612156),
 ('G#', 4.048780487804878)]

In [19]:

changing_dict = {}
for index, pair in enumerate(sorted_keys):
    changing_dict[pair[0]] = index
print(changing_dict)

df["key"].replace(changing_dict, inplace=True)

{'B': 0, 'E': 1, 'F#': 2, 'A#': 3, 'A': 4, 'F': 5, 'C#': 6, 'D#': 7, 'D': 8, 'G': 9, 'C': 10, 'G#': 11}


In [20]:
X_train, X_test, y_train, y_test, X_final, y_final = split(df, 'advanced.bin')
xgb_tree = xgb.XGBClassifier()
get_scores(xgb_tree, X_train, y_train, X_test, y_test)


Accuracy:  68.4897 %
Recall:  86.3846 %


We hoped that this operation will bring improvement. As we can observe, score is quite similar to previous ones. We decided keep it for further development anyway. 

## Hyperparameters tuning

This part is our main field for improvemt. We will try to get most of our data by changing values of the parameters. Considering tiny amount of time that xgboost needed for training, we conducted a lot of experiments changing parameters manually, seeking for best ones. Example below shows ranges between which we searched for optimal values.

In [21]:
n_estimators_n = [1, 20, 50, 90, 120]
learning_rates = [0.01, 0.03, 0.05, 0.1, 0.3, 0.7]

for n_estimator, learning_rate in zip(n_estimators_n, learning_rates):
    xgb_tree = xgb.XGBClassifier(
        n_estimators = n_estimator,
        learning_rate = learning_rate
    )
    print("N_estimators: ", n_estimator)
    print("Learning rate: ", learning_rate)
    get_scores(xgb_tree, X_train, y_train, X_test, y_test)
    print()

N_estimators:  1
Learning rate:  0.01
Accuracy:  69.6185 %
Recall:  91.1054 %

N_estimators:  20
Learning rate:  0.03
Accuracy:  70.2024 %
Recall:  91.1341 %

N_estimators:  50
Learning rate:  0.05
Accuracy:  70.6111 %
Recall:  91.422 %

N_estimators:  90
Learning rate:  0.1
Accuracy:  70.7474 %
Recall:  91.019 %

N_estimators:  120
Learning rate:  0.3
Accuracy:  68.3534 %
Recall:  86.1255 %



Best params chosen:
- n_estimators: 90
- learning rate: 0.1

## Changing the model

No matter what we did, our score didn't improve too much, that's why we decided to try a different model. Decision was made to choose SVC.

In [22]:
from sklearn.svm import SVC
svc = SVC()
get_scores(svc, X_train, y_train, X_test, y_test)

Accuracy:  70.1051 %
Recall:  95.1065 %


It appeared, that using SVC result with better basic accuracy and overall it didn't differ much from XGBoost. Disadvantage of this model is time needed to train it - while XGBoost Classifier needs maximum of 1.5 seconds to learn, SVC needs around 30. That's why our final model will be XGBoost with parameters chosen earlier.

In [23]:
cs = [0.01, 1, 10]
degrees = [2, 3]
for c in cs:
    for d in degrees:
        svc = SVC(C=c, degree=d)
        print("C: ", c, " Degree: ", d)
        get_scores(svc, X_train, y_train, X_test, y_test)
        print()

C:  0.01  Degree:  2
Accuracy:  67.6139 %
Recall:  100.0 %

C:  0.01  Degree:  3
Accuracy:  67.6139 %
Recall:  100.0 %

C:  1  Degree:  2
Accuracy:  70.1051 %
Recall:  95.1065 %

C:  1  Degree:  3
Accuracy:  70.1051 %
Recall:  95.1065 %

C:  10  Degree:  2
Accuracy:  69.7937 %
Recall:  92.228 %

C:  10  Degree:  3
Accuracy:  69.7937 %
Recall:  92.228 %



# Final model


In [24]:
final_model = xgb.XGBClassifier(
        n_estimators = 90,
        learning_rate = 0.1
    )

get_scores(xgb.XGBClassifier(n_estimators = 90, learning_rate = 0.1), X_train, y_train, X_test, y_test)
get_cross_val_scores(xgb.XGBClassifier(n_estimators = 90, learning_rate = 0.1), X_final, y_final, 5)


Accuracy:  70.7474 %
Recall:  91.019 %
Cross validation scores:  [0.68956793 0.69540677 0.6969638  0.69398482 0.69807281]


In [25]:
final_model.fit(X_train, y_train)
final_model.save_model("advanced_model.json")

Our goal of obtaining 70% accuracy is accomplished by using this hyperparameter set.

# Service

In order to make the predictions easily available, we implemented a simple service in REST methodology using Python Fastapi library. Since the project serves for academic purposes, the application was run and tested only locally; however, if we were to share our models to a wider public, we could easily deploy the service on a virtual machine in a cloud.

### Instruction

Complete implementation of the REST service can be found in Python Package `server` in our project. To run application locally, make sure you have installed all the packages from `requirements.txt` on your local environment and run module `server.py`
```
python3 server/server.py 
```
The server exposes two POST methods on the port 8000 of localhost:
* `http://localhost:8000/basic/mode` - to predict the mode attribute using the basic model
* `http://localhost:8000/advanced/mode` - to predict the mode attribute using the advanced model

Each of the methods requires a body in `json` format containing features of the track to generate prediction for. Example can be found below:

**Raw request body in json format**
```
{
  "id": "7HysCKp2SSh6LvLkcuJQkB",
  "name": "Wie weit wir gehen",
  "popularity": 49,
  "duration_ms": 179347,
  "explicit": 0,
  "id_artist": "2C7RDMSpyGZFyoSnvOeU4J",
  "release_date": "2011-06-10",
  "danceability": 0.506,
  "energy": 0.828,
  "key": 6,
  "loudness": -2.868,
  "speechiness": 0.0312,
  "acousticness": 0.0452,
  "instrumentalness": 0.0,
  "liveness": 0.206,
  "valence": 0.784,
  "tempo": 179.989,
  "time_signature": 4
}
```

**Complete example of request body and response to the REST client (Talend API Tester)**
![](images/api_response.png)

### Testing

Exposing endpoints with generated predictions makes possible to test the models with A/B testing. We evaluate two mutually exclusive hypothesis:
* *H1* - the advanced model is **not** better then the basic model
* *H2* - the advanced model is better then the basic model

To evaluate the models quality we use student's t statistic calculated using the `scipy` package.

The code below executes the following steps:
* retrieve two exclusive samples of tracks which mode is known
* test the basic model on the first sample: prepare a list of guesses, where guess equals 1 if prediction was correct and 0 otherwise
* test the advanced model on the second sample in an analogical way

**Important:** to run the tests, the server must be on

In [1]:
import pandas as pd
from scipy.stats import ttest_ind
from scipy.stats import t
import json
import requests

# Data loading

SAMPLE_SIZE = 1000
BASIC_MODEL_URL = "http://localhost:8000/basic/mode"
ADVANCED_MODEL_URL = "http://localhost:8000/advanced/mode"

df = pd.read_json("IUM23L_Zad_08_03_v2/tracks.jsonl", lines=True)
df.drop_duplicates(subset=("name", "id_artist"), inplace=True)
df = df.loc[df['mode'].notnull()]
df = df.sample(n=SAMPLE_SIZE*2)
sample_one = df.iloc[SAMPLE_SIZE:, :]
sample_two = df.iloc[:SAMPLE_SIZE, :]

# Testing

basic_model_guesses = []
for i in range(len(sample_one)):
    track = sample_one.iloc[i].squeeze().to_dict()   # retrieve track in json format
    correct_mode = track.pop("mode")                 # save the correct mode and remove it from json
    predicted_mode = requests.post(BASIC_MODEL_URL, json=track).json()["mode"]
    guess = 1 if predicted_mode == correct_mode else 0
    basic_model_guesses.append(guess)
    
advanced_model_guesses = []
for i in range(len(sample_two)):
    track = sample_two.iloc[i].squeeze().to_dict()   # retrieve track in json format
    correct_mode = track.pop("mode")                 # save the correct mode and remove it from json
    predicted_mode = requests.post(ADVANCED_MODEL_URL, json=track).json()["mode"]
    guess = 1 if predicted_mode == correct_mode else 0
    advanced_model_guesses.append(guess)

In [2]:
statistic, pvalue = ttest_ind(advanced_model_guesses, basic_model_guesses)

print(f"t-student statistic: {statistic}")
print(f"p-value: {pvalue}")

t-student statistic: 4.927203339931512
p-value: 9.02810683542512e-07


In [4]:
basic_model_accuracy = basic_model_guesses.count(1) / len(basic_model_guesses)
advanced_model_accuracy = advanced_model_guesses.count(1) / len(advanced_model_guesses)

print(f"Basic model accuracy: {basic_model_accuracy}")
print(f"Advanced model accuracy: {advanced_model_accuracy}")

Basic model accuracy: 0.619
Advanced model accuracy: 0.722


### Test result interpretation
Obtained statistic confirms that the advanced model is better than the basic model. Moreover, calculated *p-value* is very small (significantly smaller than the standard significance level $\alpha$, equal 0.005); this assures us that obtained result is not a matter of chance.  